In [0]:
data_student = [
    ("Michael", "Science",     80,  "P",  90),
    ("Nancy",   "Mathematics", 90,  "P",  None),
    ("David",   "English",     20,  "F",  80),
    ("John",    "Science",     None,"F",  None),
    ("Martin",  "Mathematics", None, None,70),
    (None,      None,          None, None, None)
]

Schema = ["name", "Subject", "Mark", "Status", "Attendance"]

df = spark.createDataFrame(data=data_student, schema=Schema)
display(df)
df.printSchema()


In [0]:
from pyspark.sql.functions import col, when, count

null_count_row = df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
])

display(null_count_row)


## Faster method

In [0]:
from pyspark.sql.functions import col, when, count

exprs = [
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]

row_df = df.select(exprs)

# convert columns → rows
result = (
    row_df
    .selectExpr("stack({0}, {1}) as (Column, Null_Count)"
                .format(len(df.columns),
                        ",".join([f"'{c}', `{c}`" for c in df.columns])))
)

display(result)
